In [39]:
import pandas as pd
import pickle
import requests
from datetime import datetime


In [40]:
btc_linear_model_path = "../artifacts/btcusdt_1d_linear_model.pkl"
eth_linear_model_path = "../artifacts/ethusdt_1d_linear_model.pkl"

with open(btc_linear_model_path, "rb") as f:
    btc_linear_model = pickle.load(f)

with open(eth_linear_model_path, "rb") as f:
    eth_linear_model = pickle.load(f)


In [41]:
btc_symbol = "BTCUSDT"
eth_symbol = "ETHUSDT"
interval = "1d"
limit = 1 # Get the latest candle

btc_url = f'https://api.binance.com/api/v3/klines?symbol={btc_symbol}&interval={interval}&limit={limit}'
eth_url = f'https://api.binance.com/api/v3/klines?symbol={eth_symbol}&interval={interval}&limit={limit}'

btc_response = requests.get(btc_url)
eth_response = requests.get(eth_url)

btc_data = btc_response.json()[0]  # Latest candlestick data
eth_data = eth_response.json()[0]  # Latest candlestick data


In [42]:
btc_data[2]

'99500.00000000'

In [43]:
btc_input_data = {
    "Open Time": int(btc_data[0]), 
    "Open": float(btc_data[1]),
    "High": float(btc_data[2]),
    "Low": float(btc_data[3]),
    "Close": float(btc_data[4]),
    "Volume": float(btc_data[5]),
    "Quote Asset Volume": float(btc_data[7]),
    "Number of Trades": int(btc_data[8]),
    "Taker Buy Base Volume": float(btc_data[9]),
    "Taker Buy Quote Volume": float(btc_data[10]),
    "Average Price": (float(btc_data[1]) + float(btc_data[4])) / 2,
    "Price Change": float(btc_data[4]) - float(btc_data[1])
}

eth_input_data = {
    "Open Time": int(eth_data[0]),  
    "Open": float(eth_data[1]),
    "High": float(eth_data[2]),
    "Low": float(eth_data[3]),
    "Close": float(eth_data[4]),
    "Volume": float(eth_data[5]),
    "Quote Asset Volume": float(eth_data[7]),
    "Number of Trades": int(eth_data[8]),
    "Taker Buy Base Volume": float(eth_data[9]),
    "Taker Buy Quote Volume": float(eth_data[10]),
    "Average Price": (float(eth_data[1]) + float(eth_data[4])) / 2,
    "Price Change": float(eth_data[4]) - float(eth_data[1])
}


In [44]:
btc_df = pd.DataFrame([btc_input_data])
eth_df = pd.DataFrame([eth_input_data])


In [45]:
btc_df

,Open Time,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Average Price,Price Change
0,1738540800000,97700.59,99500.0,91231.0,98217.95,58716.19453,5.577886e+09,8413434,28054.4964,2.666787e+09,97959.27,517.36


In [46]:
btc_df = btc_df.drop(columns=["Open Time"], errors="ignore")
eth_df = eth_df.drop(columns=["Open Time"], errors="ignore")

btc_timestamp = pd.to_datetime(btc_input_data["Open Time"], unit='ms')
eth_timestamp = pd.to_datetime(eth_input_data["Open Time"], unit='ms')

btc_df["year"] = btc_timestamp.year
btc_df["month"] = btc_timestamp.month
btc_df["day"] = btc_timestamp.day

eth_df["year"] = eth_timestamp.year
eth_df["month"] = eth_timestamp.month
eth_df["day"] = eth_timestamp.day


In [47]:
btc_linear_prediction = btc_linear_model.predict(btc_df)[0]
eth_linear_prediction = eth_linear_model.predict(eth_df)[0]


In [48]:
pd.DataFrame({"Actual BTC Price": btc_data[4], "Predicted BTC Price": btc_linear_prediction,}, index=[0])

,Actual BTC Price,Predicted BTC Price
0,98217.95000000,98217.95


In [49]:
pd.DataFrame({"Actual ETH Price": eth_data[4], "Predicted ETH Price": eth_linear_prediction,}, index=[0])

,Actual ETH Price,Predicted ETH Price
0,2681.99000000,2681.99
